# Teleoperation

In this example we'll control the Jetbot remotely with a gamepad controller connected to our web browser machine.

### Create gamepad controller

The first thing we want to do is create an instance of the ``Controller`` widget, which we'll use to drive our robot.
The ``Controller`` widget takes a ``index`` parameter, which specifies the number of the controller.  This is useful in case you
have multiple controllers attached, or some gamepads *appear* as multiple controllers.  To determine the index
of the controller you're using,

1. Visit [http://html5gamepad.com](http://html5gamepad.com).  
2. Press buttons on the gamepad you're using
3. Remember the ``index`` of the gamepad that is responding to the button presses

Next, we'll create and display our controller using that index.

In [1]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

Even if the index is correct, you may see the text ``Connect gamepad and press any button``.  That's because the gamepad hasn't
registered with this notebook yet.  Press a button and you should see the gamepad widget appear above.

### Connect gamepad controller to robot motors

Now, even though we've connected our gamepad, we haven't yet attached the controls to our robot!  The first, and most simple control
we want to attach is the motor control.  We'll connect that to the left and right vertical axes using the ``dlink`` function.  The
``dlink`` function, unlike the ``link`` function, allows us to attach a transform between the ``source`` and ``target``.  Because
the controller axes are flipped from what we think is intuitive for the motor control, we'll use a small *lambda* function to
negate the value.

> WARNING: This next cell will move the robot if you touch the gamepad controller axes!

In [46]:
from jetracer.nvidia_racecar import NvidiaRacecar
import traitlets

car = NvidiaRacecar()

car.throttle_gain = 0.2

In [47]:
car.steering_offset=0.1
car.steering = 0

In [48]:
left_link = traitlets.dlink((controller.axes[0], 'value'), (car, 'steering'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (car, 'throttle'), transform=lambda x: x)

Awesome! Our robot should now respond to our gamepad controller movements.  Now we want to view the live video feed from the camera!

In [18]:
print(len(controller.axes))

0


In [49]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=224, height=224)
# camera = USBCamera(width=224, height=224)

camera.running = True

In [52]:
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
display(camera_widget)

ClickableImageWidget(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C…